#  Version punto CERO estimación precio energía 

- Con este código puede obtenerse serie histórica de precios accesible en ESIOS (https://www.esios.ree.es/en).   
- Se precisa un ID de ESIOS. 
- En aras de confidencialidad, se asume que se ha creado un fichero de nombre ficheroDeClaves.py y dentro del mismo se ha definido la variable CLAVEdeESIOS = el ID que se posea para consultas ESIOS 
- En realidad, este notebook es "sencillo", la lógica está en las bibliotecas: bibliotecaEsios y bibliotecaGeneral  
- IMPORTANTE: Todas las horas en UTC. 
- 29 de febero de años previos, excluidos de los cálculos (ver Descripción elemental de funcionamiento)

***DESCRIPCIÓN ELEMENTAL DE FUNCIONAMIENTO:***
  
- el usuario selecciona (VER SECCIÓN PARÁMETROS de este NOTEBOOK)
    - un ***HASTA*** en horas
    - la ***FRECUENCIA*** con la que se desea la estimación de precios (EN MINUTOS)
    - un ***año desde*** ==> primer año que se consideran valores para tener en cuenta datos históricos
    - una ***tabla de IPC intraanual*** (en porcentaje) que comprende el año "desde" hasta "año actual" menos 1. ***Nota: no es preciso que se corresponda con una tabla con el IPC anual: podría ser un indice tipo "tasa anual incremento energético"- potencialmente calculable adhoc en "sesiones de modelización"- e incluso podria ponderar, además del indice de incremento de la energia, la "importancia relativa de años anteriores en la estimación de valores futuros (en definitva, que actuase como un factor que corrige la importancia relativa de años anteriores en la determinación de la predicción)***
- el usuarios lanza el programa "ahora" (en UTC)

***SECUENCIA DE CRITERIOS PARA LA DETERMINACIÓN DE PREVISIÓN DE PRECIOS:***
- se consulta en Esios los valores de precio de la energía para cada fecha/hora de los años anteriores (rango de años = (año desde, año pasado) correspondiente al horizonte de estimación. ***pte VERIFICAR: se asume que la info está sampleada cada hora para el ID que se ha utilizado***
A modo de ejemplo: 
hoy es 25 de Julio de 2000 y son las 11:48
se selecciona:
    - ***HASTA***: 2 horas 
    - ***FRECUENCIA***: 15 minutos
    - ***año desde***: 1995
    - se carga en la ***tabla de IPC*** la mejor estimación del usuario.

Se lanza el programa y: 

1.- se construye la tabla ***(!!! datos ficticios + tabla simplificada !!!!)*** hora actual + "hasta"

| hora | 1995 | 1996 | 1997 | 1998 | 1999 | 2000 |  
| :- |:- | :- |:- |:- | :- |:- | 
| 12:00 | 10  | 11 |  12 | 13 | 12.4 |  13.34  
| 13:00 | 11  | 12 |  13 | 15 | 14.4 |  13.34  
| 14:00 | 12  | 13 |  14 | 16 | 15.4 |  13.34  


2.- Se resamplea (por spline de un orden a determinar por programador) a FRECUENCIA en minutos
***(!!! datos ficticios + tabla simplificada  !!!!)***

| hora | 1995 | 1996 | 1997 | 1998 | 1999 | 2000 |  
| :- |:- | :- |:- |:- | :- |:- | 
| 12:00 | 10  | 11 |  12 | 13 | 12,4 |  13.34  
| 12:15 | 10  | 11 |  12 | 13 | 12,4 |  13.34  
| 12:30 | 10  | 11 |  12 | 13 | 12,4 |  13.34  
| 12:45 | 10  | 11 |  12 | 13 | 12,4 |  13.34  
| 13:00 | 11  | 12 |  13 | 15 | 14,4 |  13.34 
| 13:15 | 11  | 12 |  13 | 15 | 14,4 |  13.34 
| 13:30 | 11  | 12 |  13 | 15 | 14,4 |  13.34 
| 13:45 | 11  | 12 |  13 | 15 | 14,4 |  13.34 
| 14:00 | 12  | 13 |  14 | 16 | 15,4 |  13.34  

3.- Para cada reigistro: se actualiza su importe teniendo en cuenta el IPC  **recordar: el IPC puede ser un factor de correción de precio de la energía y/o factor peso importancia relativa***

| hora | 1995 | 1996 | 1997 | 1998 | 1999 | 2000 |  
| :- |:- | :- |:- |:- | :- |:- | 
| 12:00 | 11.3  | 12.45 |  12.98 | 13.34 | 12,4 |  13.34  
| 12:15 | 14.8  | 8.98 |  12.76 | 13 | 12,4 |  13.34  
| etc | etc | etc | etc | etc | etc | etc |  



# To DO

- Revisar la clase ***BajadaDatosESIOS*** con alguien con experiencia ***¿Artur?*** 
    - En general verificar si merece la pena conservar la clase definida en bibliotecaEsios para interacción con ESIOS o bien utilizar alguna otra más standarizada
    - El "ID" que ha de utilizarse para recuperar el precio de la Energía ¿correcto el tomado aquí ? 
    - Verificar que para el ID de punto anterior, su sample rate es realmente CADA HORA u otro + lo que es "peor", el sample rate cambia entre años ==> ***IMPORTANTE*** inicialmente sin impacto en la rutina de resampleo  
    - El parámetro de llamada "time_trunc"  ==> ***¿Artur?***
   
   
- Encontrar una tabla de evolución IPC de confianza y apuntar referencia. Alternativamente, podría utilizarse otro indice de corrección de precios más específico para energía o en su caso, otro valor determinado por usuarios que tenga en cuenta incluso el peso relativo de cada año  

- Pruebas por terceros

- Adicionalmente: Si se precisase que este programa formase parte de un WF que estuviera dentro de recursos de BSC, y dado que este programa exige conectarse a ESIOS (RECUERDESE que desde BSC no se puede acceder a ESIOS), la versión actual del programa podría ser muy facilmente adaptada para correr en BSC: 

  
- ***VERSIONES MÁS AVANZADAS***:   
    - En esta versión, la serie histórica misma día/hora otros años, se toma como base para el pronóstico del precio. Podríamos crear estrategias basadas en que la media es función de un número dado de horas previas DE ESTE AÑO y la varianza si función de la seria histórica Y/O pesada con valores recientes
    - Podria introducirse un "delta" en cada time step tipo lineal con t a partri del valor actual: por ejemplo: varianzaNeo = varianzaActual + factor(diferenciaDe_t_RespectoTiempoActula) * DElTA T, con objeto de introducir la heterocedasticidad de la serie 
    - Red bayesiana ???
- ***SFP*** ¡¡¡ tratamiento elemental con funcional data analysis  !!!!

- Adicionalmente, para un tratamiento de "toda la vida":
    - https://www.cienciadedatos.net/documentos/py27-forecasting-series-temporales-python-scikitlearn.html
 

In [ ]:
from __future__ import division
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
 

from bibliotecaEsios import BajadaDatosESIOS
import bibliotecaGeneral as zsan

# Chequeo preliminar
Como queda indicado, en aras de protección, se asume que existe en el directorio donde está este notebook un fichero denominado: ficheroDeClaves.py en cuyo interior existe una única clave: CLAVEdeESIOS = el id del usario

In [ ]:
try:
   from ficheroDeClaves import CLAVEdeESIOS
   id_SFP = CLAVEdeESIOS
except:
    cadenaError = '\n '
    cadenaError = '\n Con objeto de que el código sea lo más "privado" posible'
    cadenaError += '\n para ser ejecutado en modo demo '
    cadenaError += '\n crear un fichero llamado ficheroDeClaves.py'
    cadenaError += '\n y dentro del mismo definir la variable CLAVEdeESIOS = el ID que se posea para consultas ESIOS'
    raise Exception(cadenaError) 


# Parámetros

In [ ]:
horasAdelante = 10
frecuenciaSampleoMinutos = 15


AGNOINICIALCALCULOS = 2007

diccionarioAgnoIPC = {   # EN TANTO POR CIENTO
    2022: 10,
    2021: 10,    
    2020: 10,    
    2019: 10,    
    2018: 10,    
    2017: 10,    
    2016: 10,
    2015: 10,    
    2014: 10,
    2013: 10,    
    2012: 10,    
    2011: 10,
    2010: 10,
    2009: 10,
    2008: 10,    
    2007: 10    
       }

# A revisión
time_trunc = 'hours'  # Este es el parámetro de llamada de la función encontrada para interacción ESIOS
ordenSpline = 2

# Pseudoconstantes

In [ ]:
# Pseudoconstantes
ahora = datetime.now()
minutoDesde = ahora.minute
horaDesde  = ahora.hour
diaDesde  = ahora.day
mesDesde  = ahora.month
agnoActual = ahora.year

listaAgnos = list(range(AGNOINICIALCALCULOS, agnoActual))

# Chequeo: congruencia dic de IPC VS años base cálculo
Es importante verificar que para todo el rango de años considerados en el calculo (el rango de años comprendidos entre AGNOINICIALCALCULOS y agnoActual), está alimentado el diccionario de IPC's -diccionarioAgnoIPC-

In [ ]:
aux1 = [str(key) for key, value in diccionarioAgnoIPC.items()]
setAgnosEnDiccionarioIPC = sorted(set(aux1))
aux2 = [str(agno) for agno in range(AGNOINICIALCALCULOS, agnoActual)]
setRangoAgnosPrecisosParaCalculos = sorted(set(aux2))

if setRangoAgnosPrecisosParaCalculos != setAgnosEnDiccionarioIPC: 
    a =   'Revisar la definición de: \n'
    a +=  '        - diccionarioAgnoIPC \n'
    a +=  '       - y el parámetro AGNOINICIALCALCULOS \n\n'
    a +=  'CONTENIDO DE diccionarioAgnoIPC \n'
    a +=  ', '.join(setAgnosEnDiccionarioIPC)
    a +=  '\n\n'
    a +=  'Recordar que AGNOINICIALCALCULOS ha sido asignado a   ' + str(AGNOINICIALCALCULOS) + ' \n'
    a +=  '             AÑO FINAL CALCULOS (AÑO ACTUAL MENOS 1 ) ' + str(agnoActual - 1) + ' \n\n\n'
    a +=  ' ¡¡¡ AMBOS CONJUNTOS HAN DE COINCIDIR !!!'
    raise Exception(a)

# Cálculos

## Determinación dataframe de precios

### Primera columna del dataframe 
¿Por qué es diferente al resto? Porque a partir de ella se determina la columna de índice

#### 1 de 2: Leer datos desde el primer año considerado

In [ ]:
# Primera columna del dataframe de salida
inicio = datetime(AGNOINICIALCALCULOS, mesDesde, diaDesde, horaDesde, minutoDesde)       
hasta  = inicio + timedelta(hours=horasAdelante)

print("inicio: " + str(inicio) + " - hasta: " + str(hasta))

a =  BajadaDatosESIOS(id_SFP)
df = a.dataframe_lista_de_indicadores_de_esios_por_fechas([10211], inicio, hasta, time_trunc=time_trunc,
                                                         inlcuye29DeFebrero = '').filter(['value'], axis=1)

df = df.rename(columns={'value': AGNOINICIALCALCULOS})
df

In [ ]:
### 2 de 2: Cambiamos el año del indice, desde el primer año de la serie histórica al año acutal
df.index = df.index + pd.offsets.DateOffset(years=agnoActual - AGNOINICIALCALCULOS)
df

### Determinación del resto de columnas del dataframe

In [ ]:
for agno in range(AGNOINICIALCALCULOS + 1, agnoActual):
    inicio = datetime(agno, mesDesde, diaDesde, horaDesde, minutoDesde)      # desde comienzo del día
    hasta  = inicio + timedelta(hours=horasAdelante)
    aux = a.dataframe_lista_de_indicadores_de_esios_por_fechas([10211], inicio, hasta, 
                                                        time_trunc=time_trunc, inlcuye29DeFebrero = '') 
    df[agno] = list(aux['value'])

In [ ]:
df

### Correción de precios por IPC

In [ ]:
dicionarioCorreccionIPC = zsan.calculaCorrecionDePreciosPorIPC(diccionarioAgnoIPC)
dicionarioCorreccionIPC

In [ ]:
# Con los objetos que estamos manejando, la operación es trivial
for elemento in df.columns:
    df[elemento] = df[elemento] * dicionarioCorreccionIPC[elemento]
df

### Resampleo
Asumimos que la frecuencia original está en horas y se desea otro ***sample rate***

In [ ]:
print('DataFrame se resamplea a ', frecuenciaSampleoMinutos, ' minutos')
df = zsan.resampleaDataFrame(df, frecuenciaSampleoMinutos, method='polynomial', order=ordenSpline)

In [ ]:
df.head()